#Как работает сборщик мусора

При разработке проекта в Python разработчик не заботится о таких вещах, как выделение и освобождение памяти, что свойственно для разработки на C++ и C. Как только созданные объекты больше не нужны, сборщик мусора автоматически освободит память из-под них.

CPython (стандартный интерпретатор Python) использует два механизма для сборки мусора — подсчет ссылок на объекты и generational garbage collector (модуль gc в стандартной библиотеке Python).

##Алгоритм подсчета ссылок

Как известно, в Python все переменные являются ссылками на объекты. Естественно, на один объект может ссылаться несколько переменных.

In [ ]:
typedef struct _object {
    _PyObject_HEAD_EXTRA
    Py_ssize_t ob_refcnt;
    PyTypeObject *ob_type;
} PyObject;

Так выглядит структура CPython, на основе которой реализованы другие, более сложные примитивы CPython. Здесь переменная ob_refcnt — переменная, которая увеличивается каждый раз, когда на данный объект создается ссылка.

Когда эта связь пропадает, естественно, счетчик ссылки объекта уменьшается.

Каждый раз, когда счетчик ссылок становится равным нулю, запускается механизм уничтожения объекта, при этом удаляются и ссылки, которые этот объект имел к другим объектам, таким образом, уничтожение одного объекта может повлечь волну удалений других объектов.

Основной проблемой, которой обладает алгоритм подсчета ссылок, является невозможность разрешения циклических зависимостей. Эту проблему призван решить generational garbage collector.

В Python с помощью функции __sys.getrefcount()__, всегда можно узнать количество ссылок на объект.

In [2]:
import sys
lst = [1, 2, 3]
print(sys.getrefcount(lst))
i = lst
print(sys.getrefcount(i))

2
3


В примере создается список и не него ссылается переменная test_list, количество ссылок равно 1. Когда применяется функция sys.getrefcount(), количество ссылок возрастает до 2. Затем мы создаем еще одну переменную test_list_2 и присваиваем ей переменную test_list, счетчик ссылок равен 2, а вновь при применении getrefcount() становится равным 3.

Функция sys.getrefcount() обычно возвращает количество ссылок большее на единицу, чем ожидалось.

Это связано с созданием временной ссылки на аргумент, который передается в функцию.

##Generational garbage collector (GC)

Циклическая ссылка — это когда один или несколько объектов ссылаются друг на друга.

In [3]:
#Здесь список ссылается сам на себя.

a = []
a.append(a)

In [5]:
#Здесь словари ссылаются друг на друга.

a = {}
b = {}
a["b"] = b
b["a"] = a

Если вызвать метод del(), то произойдет удаление ссылок на объекты. Если бы не было GC, то объекты так бы и остались висеть в памяти, хотя и были бы недоступны для разработчика.

В отличие от подсчета ссылок, GC не срабатывает в реальном времени и запускается периодически. Для определения частоты запусков применяются встроенные эвристики. 

Generational означает «относящийся к определенному поколению» и, действительно, GC все объекты разделяет на три поколения. Изначально все объекты помещаются в первое поколение, живут там некоторое время и большинство из них очищается, остальная часть перемещается во второе поколение и потом в третье. Чем выше поколение, тем реже оно сканируется на мусор. 

Для выявления циклических ссылок GC итерирует каждый объект в поколении и временно удаляет все ссылки, на которые этот объект ссылается. После полного обхода, все обьекты, у которых счетчик ссылок меньше двух, считаются недоступными и удаляются.

GC как модуль несет для разработчика возможность управлять сборщиком мусора для циклических ссылок.

In [7]:
import gc

gc.enable() # включение сборщика мусора
gc.disable() # выключение сборщика мусора
gc.collect(generation=2) # явно инициирует проход сборщика мусора до его автоматического запуска

585